**Подсчет метрик и аналитика**

Вычисленние метрик (DAU, WAU, MAU, CR, OPU)

Ссылка на дашборд

https://redash.public.karpov.courses/dashboards/6875-analytics---main-metrics

In [5]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt 

Подсчет метрики DAU (Daily Active Users). Метрика показывает количество уникальных пользователей, взаимодействующих с продуктом за определенный день. 

In [ ]:
SELECT
    time::date AS date,
    COUNT(DISTINCT(user_id)) AS DAU
FROM
    user_actions
GROUP BY
    date
ORDER BY
    date    

In [12]:
file_path = ('data/DAU_users_2025_05_19.csv')
df = pd.read_csv(file_path)
print(df)

        date   dau
0   24/08/22   134
1   25/08/22   848
2   26/08/22  1183
3   27/08/22  1604
4   28/08/22  2145
5   29/08/22  2289
6   30/08/22  2340
7   31/08/22  2536
8   01/09/22  2816
9   02/09/22  3547
10  03/09/22  3907
11  04/09/22  4236
12  05/09/22  3480
13  06/09/22  2800
14  07/09/22  3640
15  08/09/22  4029


![Image](https://github.com/Nadya1004avt/SQL/blob/main/charts/DAU.png)

Подсчет метрики WAU (Weekly Active Users). Метрика показывает количество уникальных пользователей, взаимодействующих с продуктом в течение недели. 

In [ ]:
SELECT
    date_trunc('week', time) AS week,
    COUNT(DISTINCT(user_id)) AS WAU
FROM
    user_actions
GROUP BY
    week
ORDER BY
    week

![Image](https://github.com/Nadya1004avt/SQL/blob/main/charts/WAU.png)

In [17]:
file_path1 = ('data/WAU_users_2025_05_19.csv')
df1 = pd.read_csv(file_path1)
print(df1)

             week    wau
0  22/08/22 00:00   4692
1  29/08/22 00:00  14610
2  05/09/22 00:00  11612


Подсчет метрики MAU (Monthly Active Users). Метрика показывает количество уникальных пользователей, взаимодействующих с продуктом в течение месяца. 

In [ ]:
SELECT
    date_trunc('month', time) AS month,
    COUNT(DISTINCT(user_id)) AS MAU
FROM
    user_actions
GROUP BY
    month
ORDER BY
    month

In [16]:
file_path2 = ('data/MAU_users_2025_05_19.csv')
df2 = pd.read_csv(file_path2)
print(df2)

            month    mau
0  01/08/22 00:00   8625
1  01/09/22 00:00  18397


![Image](./charts/MAU.png)

Подсчитаем количество доставленных заказов за все время.

In [ ]:
SELECT
    COUNT(order_id) AS delivered_orders,
    time::date AS date
FROM
    courier_actions
WHERE
    action = 'deliver_order'
GROUP BY
    date
ORDER BY
    date 

In [15]:
file_path3 = ('data/Delivered_orders_2025_05_19.csv')
df3 = pd.read_csv(file_path3)
print(df3)

    delivered_orders      date
0                132  24/08/22
1               1046  25/08/22
2               1442  26/08/22
3               2134  27/08/22
4               2969  28/08/22
5               3270  29/08/22
6               3392  30/08/22
7               3399  31/08/22
8               3678  01/09/22
9               4969  02/09/22
10              5708  03/09/22
11              5991  04/09/22
12              4738  05/09/22
13              3446  06/09/22
14              4746  07/09/22
15              5556  08/09/22


![Image](./charts/Delivered_orders.png)

Вычислим сколько курьеров являются активными. 

In [ ]:
SELECT
    COUNT(DISTINCT(courier_id)) AS active_couriers,
    time::date as date
FROM
    courier_actions
GROUP BY
    date
ORDER BY
    date

In [14]:
file_path4 = ('data/Active_couriers_2025_05_19.csv')
df4 = pd.read_csv(file_path4)
print(df4)

    active_couriers      date
0                95  24/08/22
1               335  25/08/22
2               532  26/08/22
3               727  27/08/22
4               937  28/08/22
5              1028  29/08/22
6              1148  30/08/22
7              1300  31/08/22
8              1491  01/09/22
9              1749  02/09/22
10             1919  03/09/22
11             2084  04/09/22
12             2076  05/09/22
13             1923  06/09/22
14             2289  07/09/22
15             2533  08/09/22


![Image](./charts/Active_couriers.png)

Вычислим коэффициент отмен заказов. Cancel Rate(CR) - Метрика, показывающая долю отмененных заказов относительно общего числа заказов. 
Также посчитаем среднее количество заказов на пользователя. Orders per User (OPU) - показывает, сколько заказов в среднем делает один пользователь за определенный период. 

In [ ]:
WITH
t1 AS (
    SELECT
        COUNT(order_id) FILTER(WHERE action = 'cancel_order') AS cancel_orders,
        COUNT(order_id) FILTER(WHERE action = 'create_order') AS create_orders,
        COUNT(DISTINCT(user_id)) AS active_users,
        time::date AS date
    FROM
        user_actions
    GROUP BY
        date
)
SELECT
    cancel_orders::float / create_orders AS cancel_rate,
    create_orders::float / active_users AS orders_per_user,
    cancel_orders,
    create_orders,
    active_users,
    date
FROM
    t1
ORDER BY
    date    

In [13]:
file_path5 = ('data/Orders_analytics_2025_05_19.csv')
df5 = pd.read_csv(file_path5)
print(df5)

    cancel_rate  orders_per_user  cancel_orders  create_orders  active_users  \
0      0.054795         1.089552              8            146           134   
1      0.044224         1.306604             49           1108           848   
2      0.053630         1.292477             82           1529          1183   
3      0.045475         1.398379            102           2243          1604   
4      0.047317         1.468065            149           3149          2145   
5      0.053897         1.507645            186           3451          2289   
6      0.053075         1.521795            189           3561          2340   
7      0.055925         1.424290            202           3612          2536   
8      0.053645         1.383523            209           3896          2816   
9      0.051224         1.486045            270           5271          3547   
10     0.049451         1.537241            297           6006          3907   
11     0.046630         1.488432        

![Image](./charts/Orders_per_user.png)

![Image](./charts/Cancel_rate.png)